 # Multi-Step Agentic RAG with Azure AI Services
 
 This notebook demonstrates a Multi-Step Agentic RAG (Retrieval Augmented Generation) system that provides a cardiology-focused AI assistant with access to multiple data sources:

 1. **Structured Data** (Azure SQL Database) - Patient medical records and information
 2. **Unstructured Data** (Azure AI Search) - Medical guidelines from the American College of Cardiology
 3. **Web Data** (Azure AI Agent Service with Bing Grounding Tool) - Real-time web information
 
 The system uses Azure OpenAI's function/tool calling capabilities to orchestrate the retrieval process and generate comprehensive responses to medical queries.

In [3]:
#!pip install -r requirements.txt

## 1. Import Libraries and Set Up Environment
 We'll start by importing the necessary libraries and loading environment variables.


In [4]:
import os
import json
import pandas as pd
import asyncio  # noqa: F401
import aiohttp  # noqa: F401
import pyodbc  # noqa: F401
import sqlalchemy
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import BingGroundingTool
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from dotenv import load_dotenv
from openai import AzureOpenAI
from rich.console import Console
from rich.panel import Panel

load_dotenv()
console = Console()

## 2. Configure Azure Services

Next, we set up all the necessary configurations for the Azure services we'll be using.


In [5]:
# Azure OpenAI configuration
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")

print(AZURE_OPENAI_API_VERSION)

# Azure AI Search configuration
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT", "https://your-search-service.search.windows.net")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY", "your-azure-search-key")
SEARCH_INDEX_NAME = "acc-guidelines-index"

# Azure AI Project configuration for Bing Grounding
AZURE_FOUNDRY_ENDPOINT = os.getenv("AZURE_FOUNDRY_ENDPOINT", "your-azure-connection-string")
BING_CONNECTION_NAME = os.getenv("BING_CONNECTION_NAME", "fsunavalabinggrounding")

# Azure SQL connection details (for patient data)
server = os.getenv("AZURE_SQL_SERVER_NAME")
database = os.getenv("AZURE_SQL_DATABASE_NAME")
username = os.getenv("AZURE_SQL_USER_NAME")
password = os.getenv("AZURE_SQL_PASSWORD")
driver = '{ODBC Driver 17 for SQL Server}'

# Create an Azure SQL connection string
AZURE_SQL_CONNECTION_STRING = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Initialize the Azure OpenAI client
openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT  # type: ignore
)

2025-01-01-preview


## 3. Define Tool Functions

Now we'll implement the three tool functions that our agent will use to access different data sources.

### 3.1 Unstructured Data Tool: ACC Guidelines Search via Azure AI Search

This tool retrieves relevant American College of Cardiology guidelines using Azure AI Search with semantic and vector search capabilities.


In [6]:
def search_acc_guidelines(query: str) -> str:
    """
    Searches the Azure AI Search index 'acc-guidelines-index' 
    for relevant American College of Cardiology (ACC) guidelines.
    """
    credential = AzureKeyCredential(AZURE_SEARCH_KEY)
    client = SearchClient(
        endpoint=AZURE_SEARCH_ENDPOINT,
        index_name=SEARCH_INDEX_NAME,
        credential=credential,
    )
    
    results = client.search(
        search_text=query,
        vector_queries=[
            VectorizableTextQuery(
                text=query,
                k_nearest_neighbors=10,   # Adjust as needed
                fields="embedding"   # Adjust based on your index schema
            )
        ],
        query_type="semantic",
        semantic_configuration_name="default",
        search_fields=["chunk"],
        top=10,
        include_total_count=True
    )
    
    retrieved_texts = []
    for result in results:
        content_chunk = result.get("chunk", "")
        retrieved_texts.append(content_chunk)
    
    context_str = "\n".join(retrieved_texts) if retrieved_texts else "No relevant guidelines found."
    
    console.print(
        Panel(
            f"Tool Invoked: ACC Guidelines Search\nQuery: {query}",
            style="bold yellow"
        )
    )
    return context_str

 ### 3.2 Web Data Tool: Bing Web Grounding via Azure AI Agent Service
 
 This tool uses the Azure AI Agent Service with the Bing Web Grounding Tool to search for real-time information from the web. This is a significant improvement over direct Bing Search API calls, as it leverages Azure's AI Agent framework for better context and grounding and remains compliant with Bing's Terms of Use.

In [7]:
def search_bing_grounding(query: str):
    """
    Uses the Azure AI Projects client to perform a Bing search with grounding.

    Args:
        query (str): The search query

    Returns:
        str: The search results
    """
    try:
        # Create or get a project client
        project_client = AIProjectClient.from_connection_string(
            credential=DefaultAzureCredential(),
            conn_str=AZURE_FOUNDRY_ENDPOINT,
        )

        # Set up the Bing grounding tool
        bing_connection = project_client.connections.get(
            connection_name=BING_CONNECTION_NAME
        )
        conn_id = bing_connection.id
        bing = BingGroundingTool(connection_id=conn_id)

        # Create a temporary agent with the Bing tool
        with project_client:
            agent = project_client.agents.create_agent(
                model="gpt-4o",
                name="temp-bing-agent",
                instructions="Perform a search and return the results directly without additional commentary",
                tools=bing.definitions,
                headers={"x-ms-enable-preview": "true"},
            )

            # Create thread and message
            thread = project_client.agents.create_thread()
            message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content=query,
            )

            # Run the agent
            run = project_client.agents.create_and_process_run(
                thread_id=thread.id, assistant_id=agent.id
            )

            # Get the results
            if run.status == "completed":
                # Get the assistant's response
                messages = project_client.agents.list_messages(thread_id=thread.id)

                # Extract the actual message text from the response
                for msg in reversed(
                    messages.data
                ):  # Use .data to access the list of messages
                    if msg.role == "assistant" and msg.content:
                        # Extract the text value from the content
                        for content_item in msg.content:
                            if content_item.type == "text":
                                return content_item.text.value  # Return the actual text

                return "No specific weather information found."

            # Clean up
            project_client.agents.delete_agent(agent.id)

            # If we couldn't get a response
            if run.status == "failed":
                return f"Search failed: {run.last_error}"

            return "No results found."

    except Exception as e:
        return f"Error performing Bing search: {str(e)}"

### 3.3 Structured Data Tool: Patient Data Query via Azure SQL

This tool queries a SQL database containing patient medical data, transforming natural language questions into SQL queries.


In [8]:
def lookup_patient_data(query: str) -> str:
    """
    Queries the 'PatientMedicalData' table in Azure SQL and returns the results as a string.
    'query' should be a valid SQL statement.
    This version uses SQLAlchemy to create an engine, which is fully supported by pandas.read_sql.
    """
    try:
        # Construct the connection URI for SQLAlchemy
        connection_uri = (
            f"mssql+pyodbc://{username}:{password}@{server}/{database}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
        )
        engine = sqlalchemy.create_engine(connection_uri)
        
        # Use the engine in pandas.read_sql, which avoids the warning
        df = pd.read_sql(query, engine)
        if df.empty:
            return "No rows found."
        return df.to_string(index=False)
    except Exception as e:
        return f"Database error: {str(e)}"

 ## 4. Define Tools for OpenAI Function Calling
 Now we'll configure these tools to work with OpenAI's function calling feature, which allows the LLM to dynamically select and use these tools during conversation.


In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_acc_guidelines",
            "description": "Query the ACC guidelines for official cardiology recommendations. Use keywords related to cardiology conditions, treatments, or guidelines.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Keywords or specific question related to cardiology guidelines."
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_bing_grounding",
            "description": "Perform a public web search for real-time or external information using Bing Grounding. For example, 'FDA new hyperlipidemia drugs', 'recent hypertension medication approvals'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "General query to retrieve public data."
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "lookup_patient_data",
            "description": (
                "Query Azure SQL PatientMedicalData table. Schema: "
                "PatientID: INT (PK, Identity), FirstName: VARCHAR(100), LastName: VARCHAR(100), "
                "DateOfBirth: DATE, Gender: VARCHAR(20), ContactNumber: VARCHAR(100), EmailAddress: VARCHAR(100), "
                "Address: VARCHAR(255), City: VARCHAR(100), PostalCode: VARCHAR(20), Country: VARCHAR(100), "
                "MedicalCondition: VARCHAR(255), Medications: VARCHAR(255), Allergies: VARCHAR(255), BloodType: VARCHAR(10), "
                "LastVisitDate: DATE, SmokingStatus: VARCHAR(50), AlcoholConsumption: VARCHAR(50), ExerciseFrequency: VARCHAR(50), "
                "Occupation: VARCHAR(100), Height_cm: DECIMAL(5,2), Weight_kg: DECIMAL(5,2), BloodPressure: VARCHAR(20), "
                "HeartRate_bpm: INT, Temperature_C: DECIMAL(3,1), Notes: VARCHAR(MAX). Use SQL to retrieve patient data."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Valid SQL query for PatientMedicalData table to get patient information."
                    }
                },
                "required": ["query"]
            }
        }
    }
]

# Map the tool names to the actual function implementations
tool_implementations = {
    "search_acc_guidelines": search_acc_guidelines,
    "search_bing_grounding": search_bing_grounding,
    "lookup_patient_data": lookup_patient_data,
}

## 5. Define System Prompt for the Agent

The system prompt sets the context and instructions for the Agent, defining its role and capabilities.


In [10]:
# ----------------------------
# System Prompt for the Agent
# ----------------------------
SYSTEM_PROMPT = (
    "You are a cardiology-focused AI assistant with access to three tools:\n"
    "1) 'lookup_patient_data' for querying patient records from Azure SQL.\n"
    "2) 'search_acc_guidelines' for official ACC guidelines.\n"
    "3) 'search_bing_grounding' for real-time public information.\n\n"
    "You can call these tools in any order, multiple times if needed, to gather all the context.\n"
    "Stop calling tools only when you have enough information to provide a final, cohesive answer.\n"
    "Then output your final answer to the user."
)


## 6. Implement the Multi-Step Agent

 Now we'll create the agent that orchestrates the entire process. This agent will:

1.  Receive a user query
2.  Decide which tools to call and in what order
3.  Parse the results from each tool
4.  Synthesize a comprehensive answer

The agent can make multiple calls to different tools before providing a final answer, allowing for complex, multi-step reasoning.


In [11]:
def run_multi_step_agent(user_query: str, max_steps: int = 3):
    """
    A multi-step agent that orchestrates tool selection and execution.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query}
    ]

    for step in range(max_steps):
        console.print(Panel(f"**Step {step+1}**: Starting step {step+1}", title="Step Start", style="bold cyan"))
        
        try:
            # Log current message state for debugging
            console.print(Panel(f"Messages before API call:\n{json.dumps(messages, indent=2)}", title="Current Messages", style="dim"))
            
            # Make API call to OpenAI
            response = openai_client.chat.completions.create(
                model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                messages=messages, # type: ignore
                tools=tools, # type: ignore
                tool_choice="auto",
                max_tokens=8000,
            )
            response_message = response.choices[0].message
            
            # Debug the raw response
            console.print(Panel(f"Raw response message:\n{response_message}", title="Raw Response", style="dim"))
            
            # Check if the response contains tool calls
            if response_message.tool_calls:
                # Create a properly structured assistant message with tool_calls
                messages.append({
                    "role": "assistant",
                    "content": response_message.content,  # This might be None when tool_calls are present
                    "tool_calls": [
                        {
                            "id": tool_call.id,
                            "type": "function",
                            "function": {
                                "name": tool_call.function.name,
                                "arguments": tool_call.function.arguments
                            }
                        } for tool_call in response_message.tool_calls
                    ] # type: ignore
                })
                
                # Now handle each tool call separately
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    tool_call_id = tool_call.id  # Extract the exact tool_call_id
                    arguments_str = tool_call.function.arguments
                    
                    try:
                        function_args = json.loads(arguments_str) if arguments_str.strip() else {}
                    except json.JSONDecodeError:
                        function_args = {}
                        console.print(Panel("Warning: Could not decode tool call arguments", title="JSONDecodeError", style="bold red"))
                    
                    # Ensure the 'query' key is present
                    if "query" not in function_args or not function_args["query"]:
                        function_args["query"] = user_query
                    
                    console.print(Panel(
                        f"**Step {step+1}**: LLM calls tool [bold]{function_name}[/bold]\n\n"
                        f"**Arguments**:\n{json.dumps(function_args, indent=2)}",
                        title="Tool Call",
                        style="bold blue"
                    ))
                    
                    # Call the appropriate tool function
                    tool_fn = tool_implementations.get(function_name)
                    if tool_fn is None:
                        tool_output = f"[Error] No implementation for tool '{function_name}'."
                    else:
                        tool_output = tool_fn(**function_args)
                    
                    # Add the tool response with EXACTLY the same tool_call_id 
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call_id,  # CRITICAL: Use exactly the same ID
                        "name": function_name,
                        "content": str(tool_output)
                    })
                    
                    console.print(Panel(f"Tool output added to messages:\n{str(tool_output)}", title="Tool Response Added", style="green"))
            else:
                # No tool calls, just add the assistant's message and return the final answer
                messages.append({
                    "role": "assistant",
                    "content": response_message.content # type: ignore
                })
                
                # Display the final answer
                final_answer = response_message.content
                console.print(Panel(final_answer, title="Final Answer", style="bold green", border_style="yellow")) # type: ignore
                return  # Successfully received final answer
        
        except Exception as e:
            console.print(Panel(f"Error during step {step+1}: {str(e)}", title="Error", style="bold red"))
            # Print the current message structure to help with debugging
            console.print(Panel(f"Current message structure:\n{json.dumps(messages, indent=2)}", title="Message Structure at Error", style="red"))
            raise  # Re-raise the exception for further debugging

    # If we reach the maximum steps without a final answer, show a warning
    console.print(Panel("Max steps reached without a final answer. Stopping.", title="Warning", style="bold red"))
    return

 ## 7. Execute the Agent with a Sample Query
 
 Now let's run the agent with a cardiology-related question to see how it works. The agent will decide which tools to use and in what order to provide the most comprehensive answer.

In [12]:
user_question_1 = "What does the ACC recommend as first-line therapy for hypertension in elderly patients?"
run_multi_step_agent(user_question_1)


╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Messages ────────────────────────────────────────────────╮
│ Messages before API call:                                                                                       │
│ [                                                                                                               │
│   {                                                                                                             │
│     "role": "system",                                                                                           │
│     "content": "You are a cardiology-focused AI assistant with access to three tools:\n1) 'lookup_patient_data' │
│ for querying patient records from Azure SQL.\n2) 'search_acc_guidelines' for official ACC guidelines.\n3)       │
│ 'search_bing_grounding' for real-time public information.\n\nYou can call these tools in any order, multiple    │
│ times if needed, to gather all the context.\nStop calling tools only when you have enough information to        │
│ provide a final, cohesive answer.\nThen output your final answer to the user."                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "user",                                                                                             │
│     "content": "What does the ACC recommend as first-line therapy for hypertension in elderly patients?"        │
│   }                                                                                                             │
│ ]                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ Raw response message:                                                                                           │
│ ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None,             │
│ tool_calls=[ChatCompletionMessageToolCall(id='call_yVXBesnr1g0Ttb2OmZkWUH5z',                                   │
│ function=Function(arguments='{"query":"first-line therapy for hypertension in elderly patients"}',              │
│ name='search_acc_guidelines'), type='function')], annotations=[])                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Tool Call ───────────────────────────────────────────────────╮
│ **Step 1**: LLM calls tool search_acc_guidelines                                                                │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "first-line therapy for hypertension in elderly patients"                                            │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── Error ─────────────────────────────────────────────────────╮
│ Error during step 1: () The given API key doesn't match service's internal, primary or secondary keys.          │
│ Code:                                                                                                           │
│ Message: The given API key doesn't match service's internal, primary or secondary keys.                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Message Structure at Error ───────────────────────────────────────────╮
│ Current message structure:                                                                                      │
│ [                                                                                                               │
│   {                                                                                                             │
│     "role": "system",                                                                                           │
│     "content": "You are a cardiology-focused AI assistant with access to three tools:\n1) 'lookup_patient_data' │
│ for querying patient records from Azure SQL.\n2) 'search_acc_guidelines' for official ACC guidelines.\n3)       │
│ 'search_bing_grounding' for real-time public information.\n\nYou can call these tools in any order, multiple    │
│ times if needed, to gather all the context.\nStop calling tools only when you have enough information to        │
│ provide a final, cohesive answer.\nThen output your final answer to the user."                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "user",                                                                                             │
│     "content": "What does the ACC recommend as first-line therapy for hypertension in elderly patients?"        │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "assistant",                                                                                        │
│     "content": null,                                                                                            │
│     "tool_calls": [                                                                                             │
│       {                                                                                                         │
│         "id": "call_yVXBesnr1g0Ttb2OmZkWUH5z",                                                                  │
│         "type": "function",                                                                                     │
│         "function": {                                                                                           │
│           "name": "search_acc_guidelines",                                                                      │
│           "arguments": "{\"query\":\"first-line therapy for hypertension in elderly patients\"}"                │
│         }                                                                                                       │
│       }                                                                                                         │
│     ]                                                                                                           │
│   }                                                                                                             │
│ ]                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

HttpResponseError: () The given API key doesn't match service's internal, primary or secondary keys.
Code: 
Message: The given API key doesn't match service's internal, primary or secondary keys.

In [82]:
user_question_2 = "Any latest FDA approvals or clinical trial facts related to Ozempic and how it impacts cardiovascular diseases?"
run_multi_step_agent(user_question_2)


╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Messages ────────────────────────────────────────────────╮
│ Messages before API call:                                                                                       │
│ [                                                                                                               │
│   {                                                                                                             │
│     "role": "system",                                                                                           │
│     "content": "You are a cardiology-focused AI assistant with access to three tools:\n1) 'lookup_patient_data' │
│ for querying patient records from Azure SQL.\n2) 'search_acc_guidelines' for official ACC guidelines.\n3)       │
│ 'search_bing_grounding' for real-time public information.\n\nYou can call these tools in any order, multiple    │
│ times if needed, to gather all the context.\nStop calling tools only when you have enough information to        │
│ provide a final, cohesive answer.\nThen output your final answer to the user."                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "user",                                                                                             │
│     "content": "Any latest FDA approvals or clinical trial facts related to Ozempic and how it impacts          │
│ cardiovascular diseases?"                                                                                       │
│   }                                                                                                             │
│ ]                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ Raw response message:                                                                                           │
│ ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None,             │
│ tool_calls=[ChatCompletionMessageToolCall(id='call_iAOtTON8Nhk8hU6KIbIQOeoz',                                   │
│ function=Function(arguments='{"query": "latest FDA approvals Ozempic cardiovascular disease"}',                 │
│ name='search_bing_grounding'), type='function'),                                                                │
│ ChatCompletionMessageToolCall(id='call_3jzH1RKe67FlErM2jzL3LJ3h', function=Function(arguments='{"query":        │
│ "Ozempic clinical trials cardiovascular outcomes"}', name='search_bing_grounding'), type='function')])          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Tool Call ───────────────────────────────────────────────────╮
│ **Step 1**: LLM calls tool search_bing_grounding                                                                │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "latest FDA approvals Ozempic cardiovascular disease"                                                │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Tool Response Added ──────────────────────────────────────────────╮
│ Tool output added to messages:                                                                                  │
│ On January 28, 2025, the FDA approved a new indication for Ozempic (semaglutide), making it the only GLP-1      │
│ receptor agonist approved to reduce the risk of kidney disease worsening, kidney failure (end-stage kidney      │
│ disease), and death due to cardiovascular disease in adults with type 2 diabetes and chronic kidney disease     │
│ (CKD)【0†source】【2†source】. This approval is based on the results from the FLOW phase 3b kidney outcomes     │
│ trial which demonstrated a statistically significant reduction in risks associated with these                   │
│ conditions【2†source】.                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────── Tool Response Added ──────────────────────────────────────────────╮
│ Tool output added to messages:                                                                                  │
│ The SUSTAIN 6 clinical trial evaluated the cardiovascular outcomes of Ozempic (semaglutide) in patients with    │
│ type 2 diabetes and high cardiovascular risk. The trial showed a significant reduction in the risk of major     │
│ adverse cardiovascular events (MACE), which includes cardiovascular death, non-fatal myocardial infarction, and │
│ non-fatal stroke, with a 26% risk reduction compared to placebo. The reduction was mainly driven by fewer       │
│ non-fatal myocardial infarctions and strokes【7†source】.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 2**: Starting step 2                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Messages ────────────────────────────────────────────────╮
│ Messages before API call:                                                                                       │
│ [                                                                                                               │
│   {                                                                                                             │
│     "role": "system",                                                                                           │
│     "content": "You are a cardiology-focused AI assistant with access to three tools:\n1) 'lookup_patient_data' │
│ for querying patient records from Azure SQL.\n2) 'search_acc_guidelines' for official ACC guidelines.\n3)       │
│ 'search_bing_grounding' for real-time public information.\n\nYou can call these tools in any order, multiple    │
│ times if needed, to gather all the context.\nStop calling tools only when you have enough information to        │
│ provide a final, cohesive answer.\nThen output your final answer to the user."                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "user",                                                                                             │
│     "content": "Any latest FDA approvals or clinical trial facts related to Ozempic and how it impacts          │
│ cardiovascular diseases?"                                                                                       │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "assistant",                                                                                        │
│     "content": null,                                                                                            │
│     "tool_calls": [                                                                                             │
│       {                                                                                                         │
│         "id": "call_iAOtTON8Nhk8hU6KIbIQOeoz",                                                                  │
│         "type": "function",                                                                                     │
│         "function": {                                                                                           │
│           "name": "search_bing_grounding",                                                                      │
│           "arguments": "{\"query\": \"latest FDA approvals Ozempic cardiovascular disease\"}"                   │
│         }                                                                                                       │
│       },                                                                                                        │
│       {                                                                                                         │
│         "id": "call_3jzH1RKe67FlErM2jzL3LJ3h",                                                                  │
│         "type": "function",                                                                                     │
│         "function": {                                                                                           │
│           "name": "search_bing_grounding",                                                                      │
│           "arguments": "{\"query\": \"Ozempic clinical trials cardiovascular outcomes\"}"                       │
│         }                                             

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ Raw response message:                                                                                           │
│ ChatCompletionMessage(content='Ozempic (semaglutide) has seen significant advancements in its FDA approval and  │
│ clinical trial outcomes related to cardiovascular diseases:\n\n1. **FDA Approval**: On January 28, 2025, the    │
│ FDA granted a new indication for Ozempic, making it the only GLP-1 receptor agonist approved specifically to    │
│ reduce the risk of kidney disease worsening, kidney failure (end-stage kidney disease), and death due to        │
│ cardiovascular disease in adults with type 2 diabetes and chronic kidney disease (CKD). This approval was based │
│ on the findings from the FLOW phase 3b kidney outcomes trial, which demonstrated a significant reduction in the │
│ risks associated with these conditions.\n\n2. **Clinical Trials**: The SUSTAIN 6 clinical trial evaluated the   │
│ cardiovascular outcomes of Ozempic in patients with type 2 diabetes who are at high cardiovascular risk. The    │
│ results revealed a 26% reduction in the risk of major adverse cardiovascular events (MACE)—which includes       │
│ cardiovascular death, non-fatal myocardial infarction, and non-fatal stroke—when compared to a placebo. The     │
│ risk reduction was primarily attributed to fewer occurrences of non-fatal myocardial infarctions and strokes in │
│ the Ozempic group.\n\nThese findings suggest that Ozempic not only aids in glycemic control but also has        │
│ protective benefits against cardiovascular events in diabetic patients at risk.', refusal=None,                 │
│ role='assistant', audio=None, function_call=None, tool_calls=None)                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Ozempic (semaglutide) has seen significant advancements in its FDA approval and clinical trial outcomes related │
│ to cardiovascular diseases:                                                                                     │
│                                                                                                                 │
│ 1. **FDA Approval**: On January 28, 2025, the FDA granted a new indication for Ozempic, making it the only      │
│ GLP-1 receptor agonist approved specifically to reduce the risk of kidney disease worsening, kidney failure     │
│ (end-stage kidney disease), and death due to cardiovascular disease in adults with type 2 diabetes and chronic  │
│ kidney disease (CKD). This approval was based on the findings from the FLOW phase 3b kidney outcomes trial,     │
│ which demonstrated a significant reduction in the risks associated with these conditions.                       │
│                                                                                                                 │
│ 2. **Clinical Trials**: The SUSTAIN 6 clinical trial evaluated the cardiovascular outcomes of Ozempic in        │
│ patients with type 2 diabetes who are at high cardiovascular risk. The results revealed a 26% reduction in the  │
│ risk of major adverse cardiovascular events (MACE)—which includes cardiovascular death, non-fatal myocardial    │
│ infarction, and non-fatal stroke—when compared to a placebo. The risk reduction was primarily attributed to     │
│ fewer occurrences of non-fatal myocardial infarctions and strokes in the Ozempic group.                         │
│                                                                                                                 │
│ These findings suggest that Ozempic not only aids in glycemic control but also has protective benefits against  │
│ cardiovascular events in diabetic patients at risk.                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [83]:
user_question_3 = "How many patients have Hypertension and are prescribed Lisinopril?"
# The agent should generate a valid SQL query, for example:
## Note, the answer will depend on the actual data in the database (it should return 1071!)
run_multi_step_agent(user_question_3)


╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Messages ────────────────────────────────────────────────╮
│ Messages before API call:                                                                                       │
│ [                                                                                                               │
│   {                                                                                                             │
│     "role": "system",                                                                                           │
│     "content": "You are a cardiology-focused AI assistant with access to three tools:\n1) 'lookup_patient_data' │
│ for querying patient records from Azure SQL.\n2) 'search_acc_guidelines' for official ACC guidelines.\n3)       │
│ 'search_bing_grounding' for real-time public information.\n\nYou can call these tools in any order, multiple    │
│ times if needed, to gather all the context.\nStop calling tools only when you have enough information to        │
│ provide a final, cohesive answer.\nThen output your final answer to the user."                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "user",                                                                                             │
│     "content": "How many patients have Hypertension and are prescribed Lisinopril?"                             │
│   }                                                                                                             │
│ ]                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ Raw response message:                                                                                           │
│ ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None,             │
│ tool_calls=[ChatCompletionMessageToolCall(id='call_Oh3Ab9yhY2Q36sTIaJ5PWaAm',                                   │
│ function=Function(arguments='{"query":"SELECT COUNT(*) FROM PatientMedicalData WHERE MedicalCondition LIKE      │
│ \'%Hypertension%\' AND Medications LIKE \'%Lisinopril%\'"}', name='lookup_patient_data'), type='function')])    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Tool Call ───────────────────────────────────────────────────╮
│ **Step 1**: LLM calls tool lookup_patient_data                                                                  │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "SELECT COUNT(*) FROM PatientMedicalData WHERE MedicalCondition LIKE '%Hypertension%' AND            │
│ Medications LIKE '%Lisinopril%'"                                                                                │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Tool Response Added ──────────────────────────────────────────────╮
│ Tool output added to messages:                                                                                  │
│                                                                                                                 │
│ 1071                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 2**: Starting step 2                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Messages ────────────────────────────────────────────────╮
│ Messages before API call:                                                                                       │
│ [                                                                                                               │
│   {                                                                                                             │
│     "role": "system",                                                                                           │
│     "content": "You are a cardiology-focused AI assistant with access to three tools:\n1) 'lookup_patient_data' │
│ for querying patient records from Azure SQL.\n2) 'search_acc_guidelines' for official ACC guidelines.\n3)       │
│ 'search_bing_grounding' for real-time public information.\n\nYou can call these tools in any order, multiple    │
│ times if needed, to gather all the context.\nStop calling tools only when you have enough information to        │
│ provide a final, cohesive answer.\nThen output your final answer to the user."                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "user",                                                                                             │
│     "content": "How many patients have Hypertension and are prescribed Lisinopril?"                             │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "assistant",                                                                                        │
│     "content": null,                                                                                            │
│     "tool_calls": [                                                                                             │
│       {                                                                                                         │
│         "id": "call_Oh3Ab9yhY2Q36sTIaJ5PWaAm",                                                                  │
│         "type": "function",                                                                                     │
│         "function": {                                                                                           │
│           "name": "lookup_patient_data",                                                                        │
│           "arguments": "{\"query\":\"SELECT COUNT(*) FROM PatientMedicalData WHERE MedicalCondition LIKE        │
│ '%Hypertension%' AND Medications LIKE '%Lisinopril%'\"}"                                                        │
│         }                                                                                                       │
│       }                                                                                                         │
│     ]                                                                                                           │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "tool",                                                                                             │
│     "tool_call_id": "call_Oh3Ab9yhY2Q36sTIaJ5PWaAm",                                                            │
│     "name": "lookup_patient_data",                                                                              │
│     "content": "    \n1071"                           

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ Raw response message:                                                                                           │
│ ChatCompletionMessage(content='There are 1,071 patients who have Hypertension and are prescribed Lisinopril.',  │
│ refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ There are 1,071 patients who have Hypertension and are prescribed Lisinopril.                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [84]:
run_multi_step_agent("I have a 79-year-old patient named Gloria Paul with hyperlipidemia. She's on Atorvastatin. Can you confirm her medical details from the database, check the ACC guidelines for hyperlipidemia, and see if there are any new medication updates from the FDA as of Feb 2025? Then give me a summary.")

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 1**: Starting step 1                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Messages ────────────────────────────────────────────────╮
│ Messages before API call:                                                                                       │
│ [                                                                                                               │
│   {                                                                                                             │
│     "role": "system",                                                                                           │
│     "content": "You are a cardiology-focused AI assistant with access to three tools:\n1) 'lookup_patient_data' │
│ for querying patient records from Azure SQL.\n2) 'search_acc_guidelines' for official ACC guidelines.\n3)       │
│ 'search_bing_grounding' for real-time public information.\n\nYou can call these tools in any order, multiple    │
│ times if needed, to gather all the context.\nStop calling tools only when you have enough information to        │
│ provide a final, cohesive answer.\nThen output your final answer to the user."                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "user",                                                                                             │
│     "content": "I have a 79-year-old patient named Gloria Paul with hyperlipidemia. She's on Atorvastatin. Can  │
│ you confirm her medical details from the database, check the ACC guidelines for hyperlipidemia, and see if      │
│ there are any new medication updates from the FDA as of Feb 2025? Then give me a summary."                      │
│   }                                                                                                             │
│ ]                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ Raw response message:                                                                                           │
│ ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None,             │
│ tool_calls=[ChatCompletionMessageToolCall(id='call_wDYynvTCJlXDSeGHihDW3rof',                                   │
│ function=Function(arguments='{"query": "SELECT * FROM PatientMedicalData WHERE FirstName=\'Gloria\' AND         │
│ LastName=\'Paul\'"}', name='lookup_patient_data'), type='function'),                                            │
│ ChatCompletionMessageToolCall(id='call_AcZAtI7rblAPdmxzoC6nxGgg', function=Function(arguments='{"query":        │
│ "hyperlipidemia"}', name='search_acc_guidelines'), type='function'),                                            │
│ ChatCompletionMessageToolCall(id='call_Wj83hyOr277uKGy1oSceoOi9', function=Function(arguments='{"query": "FDA   │
│ new hyperlipidemia drugs February 2025"}', name='search_bing_grounding'), type='function')])                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Tool Call ───────────────────────────────────────────────────╮
│ **Step 1**: LLM calls tool lookup_patient_data                                                                  │
│                                                                                                                 │
│ **Arguments**:                                                                                                  │
│ {                                                                                                               │
│   "query": "SELECT * FROM PatientMedicalData WHERE FirstName='Gloria' AND LastName='Paul'"                      │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Tool Response Added ──────────────────────────────────────────────╮
│ Tool output added to messages:                                                                                  │
│  PatientID FirstName LastName DateOfBirth Gender ContactNumber        EmailAddress                              │
│ Address        City PostalCode Country MedicalCondition  Medications Allergies BloodType LastVisitDate          │
│ SmokingStatus AlcoholConsumption ExerciseFrequency            Occupation  Height_cm  Weight_kg BloodPressure    │
│ HeartRate_bpm  Temperature_C                                                                                    │
│ Notes                                                                                                           │
│          1    Gloria     Paul  1944-07-15  Other  428-674-1336 keymary@example.com 62908 Fuentes                │
│ Cove\nLewisberg, MN 13491 Buchananton      59963   India   Hyperlipidemia Atorvastatin      None        O+      │
│ 2024-03-26  Never Smoker      Light drinker  1-2 times a week Administrator, sports      176.2       77.7       │
│ 120/79 mmHg             90           37.6 Affect idea turn either see six. Suggest cultural change nice travel  │
│ year parent. Happen energy level anyone position. Few available thought world development.                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────── Tool Response Added ──────────────────────────────────────────────╮
│ Tool output added to messages:                                                                                  │
│ In February 2025, the FDA accepted the Biologics License Application for Lerodalcibep, a novel third-generation │
│ PCSK9 inhibitor developed by LIB Therapeutics. This drug aims to reduce low-density lipoprotein cholesterol     │
│ (LDL-C) for patients with atherosclerotic cardiovascular disease (ASCVD) and primary hyperlipidemia.            │
│ Lerodalcibep is designed to be a more convenient, once-monthly, subcutaneous injection that does not require    │
│ refrigeration. The FDA's action date for the review is set for December 12, 2025【0†source】【1†source】.       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Step Start ───────────────────────────────────────────────────╮
│ **Step 2**: Starting step 2                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Current Messages ────────────────────────────────────────────────╮
│ Messages before API call:                                                                                       │
│ [                                                                                                               │
│   {                                                                                                             │
│     "role": "system",                                                                                           │
│     "content": "You are a cardiology-focused AI assistant with access to three tools:\n1) 'lookup_patient_data' │
│ for querying patient records from Azure SQL.\n2) 'search_acc_guidelines' for official ACC guidelines.\n3)       │
│ 'search_bing_grounding' for real-time public information.\n\nYou can call these tools in any order, multiple    │
│ times if needed, to gather all the context.\nStop calling tools only when you have enough information to        │
│ provide a final, cohesive answer.\nThen output your final answer to the user."                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "user",                                                                                             │
│     "content": "I have a 79-year-old patient named Gloria Paul with hyperlipidemia. She's on Atorvastatin. Can  │
│ you confirm her medical details from the database, check the ACC guidelines for hyperlipidemia, and see if      │
│ there are any new medication updates from the FDA as of Feb 2025? Then give me a summary."                      │
│   },                                                                                                            │
│   {                                                                                                             │
│     "role": "assistant",                                                                                        │
│     "content": null,                                                                                            │
│     "tool_calls": [                                                                                             │
│       {                                                                                                         │
│         "id": "call_wDYynvTCJlXDSeGHihDW3rof",                                                                  │
│         "type": "function",                                                                                     │
│         "function": {                                                                                           │
│           "name": "lookup_patient_data",                                                                        │
│           "arguments": "{\"query\": \"SELECT * FROM PatientMedicalData WHERE FirstName='Gloria' AND             │
│ LastName='Paul'\"}"                                                                                             │
│         }                                                                                                       │
│       },                                                                                                        │
│       {                                                                                                         │
│         "id": "call_AcZAtI7rblAPdmxzoC6nxGgg",                                                                  │
│         "type": "function",                                                                                     │
│         "function": {                                                                                           │
│           "name": "search_acc_guidelines",            

╭───────────────────────────────────────────────── Raw Response ──────────────────────────────────────────────────╮
│ Raw response message:                                                                                           │
│ ChatCompletionMessage(content="### Patient Summary\n- **Name:** Gloria Paul\n- **Date of Birth:** July 15, 1944 │
│ (Age: 79)\n- **Gender:** Other\n- **Medical Condition:** Hyperlipidemia\n- **Medications:** Atorvastatin\n-     │
│ **Allergies:** None\n- **Blood Type:** O+\n- **Last Visit Date:** March 26, 2024\n- **Smoking Status:** Never   │
│ Smoker\n- **Alcohol Consumption:** Light drinker (1-2 times a week)\n- **Exercise Frequency:** Light\n-         │
│ **Occupation:** Administrator, sports\n- **Height:** 176.2 cm\n- **Weight:** 77.7 kg\n- **Blood Pressure:**     │
│ 120/79 mmHg\n- **Heart Rate:** 90 bpm\n- **Notes:** Descriptions of personal insights and lifestyle changes are │
│ noted.\n\n### ACC Guidelines for Hyperlipidemia\n1. **Statin Therapy:**\n   - **Class I**: Treatment with       │
│ statin medication is recommended for patients with significant atherosclerotic diseases to reduce LDL           │
│ cholesterol levels below 100 mg/dL.\n   - **Class IIa**: It is reasonable to aim for LDL cholesterol levels     │
│ near or below 70 mg/dL in patients who have had ischemic stroke.\n   - If LDL targets are not achieved with     │
│ statins, intensifying therapy with additional medications may be appropriate.\n\n2. **Lifestyle                 │
│ Modifications**: Intensifying lifestyle changes, including diet and exercise, is essential for managing         │
│ hyperlipidemia.\n\n3. **Comorbidities**: Special attention should be given to hypertension and diabetes as they │
│ frequently co-occur with hyperlipidemia, which complicates clinical management.\n\n### New Medication Updates   │
│ (As of February 2025)\n- The **FDA accepted the Biologics License Application for Lerodalcibep**, a novel PCSK9 │
│ inhibitor designed for patients with atherosclerotic cardiovascular disease (ASCVD) and primary hyperlipidemia. │
│ \n- **Features**: Lerodalcibep is a once-monthly subcutaneous injection that does not require refrigeration,    │
│ making it a potentially more convenient treatment option.\n- The FDA's review action date for this medication   │
│ is December 12, 2025.\n\n### Summary\nGloria Paul is a 79-year-old woman effectively managed on atorvastatin    │
│ for hyperlipidemia, meeting current ACC guidelines emphasizing the use of statins to achieve lower LDL          │
│ cholesterol targets while recommending lifestyle modifications. Notably, the recent approval process for a new  │
│ drug, Lerodalcibep, may offer further treatment options for patients with similar conditions. Always ensure     │
│ adherence to guidelines when managing complex medical cases, especially in elderly patients.", refusal=None,    │
│ role='assistant', audio=None, function_call=None, tool_calls=None)                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ ### Patient Summary                                                                                             │
│ - **Name:** Gloria Paul                                                                                         │
│ - **Date of Birth:** July 15, 1944 (Age: 79)                                                                    │
│ - **Gender:** Other                                                                                             │
│ - **Medical Condition:** Hyperlipidemia                                                                         │
│ - **Medications:** Atorvastatin                                                                                 │
│ - **Allergies:** None                                                                                           │
│ - **Blood Type:** O+                                                                                            │
│ - **Last Visit Date:** March 26, 2024                                                                           │
│ - **Smoking Status:** Never Smoker                                                                              │
│ - **Alcohol Consumption:** Light drinker (1-2 times a week)                                                     │
│ - **Exercise Frequency:** Light                                                                                 │
│ - **Occupation:** Administrator, sports                                                                         │
│ - **Height:** 176.2 cm                                                                                          │
│ - **Weight:** 77.7 kg                                                                                           │
│ - **Blood Pressure:** 120/79 mmHg                                                                               │
│ - **Heart Rate:** 90 bpm                                                                                        │
│ - **Notes:** Descriptions of personal insights and lifestyle changes are noted.                                 │
│                                                                                                                 │
│ ### ACC Guidelines for Hyperlipidemia                                                                           │
│ 1. **Statin Therapy:**                                                                                          │
│    - **Class I**: Treatment with statin medication is recommended for patients with significant atherosclerotic │
│ diseases to reduce LDL cholesterol levels below 100 mg/dL.                                                      │
│    - **Class IIa**: It is reasonable to aim for LDL cholesterol levels near or below 70 mg/dL in patients who   │
│ have had ischemic stroke.                                                                                       │
│    - If LDL targets are not achieved with statins, intensifying therapy with additional medications may be      │
│ appropriate.                                                                                                    │
│                                                                                                                 │
│ 2. **Lifestyle Modifications**: Intensifying lifestyle changes, including diet and exercise, is essential for   │
│ managing hyperlipidemia.                                                                                        │
│                                                                                                                 │
│ 3. **Comorbidities**: Special attention should be given to hypertension and diabetes as they frequently         │
│ co-occur with hyperlipidemia, which complicates clinical management.                                            │
│                                                       